# Exploratory Data Analysis of Multiversion Concurrency Control Conflict in Hyperledger Fabric
In Hyperledger-Fabric each transaction is first executed, then ordered by data dependencies, then validated against the current state of the blockchain. Because of this architecture transactions can be executed in parallel, however if a key is written to, its version  number is updated and all transactions that have not yet been validated and read the previous version number, get invalidated. They receive the validation code of MVCC_READ_CONFILCT.

If a lot of transactions receive this code, it will negatively affect performance of the system. This notebook aims to uncover the patterns that cause a lot of these conflicts, so the chaincode can be rewritten in a way that avoids this.

This notebook depends on a running explorer database. With the provided shell script, a database can be run locally, with the sample data provided in the data folder.

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
# import bamboolib as bam
import plotly.express as px
import json
import re
from datetime import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.io as pio
pio.renderers.default = "iframe"
import networkx as nx
import swifter
from graphdatascience import GraphDataScience

In [ ]:
# function to extract the asset type from the composite-keys
def getAssetType(key):
    parts=key.split("||")
    if len(parts)>1:
        return parts[1]
    return parts[0]
# function to assign -1 to mvccs, +1 to valid, 0 to all else
def gradeValidation(code):
    if code=='MVCC_READ_CONFLICT':
        return -1
    elif code=='VALID':
        return +1
    else:
        return 0
# function to find keys in the rwsets' jsons
def findKeys(string):
    pat = r"(?<='key': ').+?(?=')"
    return re.findall(pat,string)
# function to simulate complex keys, if the keys in the chaincode represent different asset types, but complex keys were not used
def simulateCompKeys(key,asset_type_prefix):
    k = str(key)
    if(asset_type_prefix in k):
        parts = k.split(asset_type_prefix)
        return '\x00' + k[0]+ asset_type_prefix + '\x00' + parts[1] + '\x00'
    else:
        return str(key)
# function to determine the tx level in a block
def blockLevelTxNum(row):
    if(row['blockid'] ==row['prev_blockid']):
        return row['prev_lvl']+1
    return 0
# function to check if a column contains a series of integers without missing any between the min and max 
def continuityCheck(df,columname):
    tmp_df = df.sort_values(by=columname,ascending=True)
    check = {'col':tmp_df[columname],'check':tmp_df[columname].diff()}
    check = pd.DataFrame(check)
    check = check.loc[check['check']>abs(1)]
    missing = check['check'].sum()
    print("Total missing from " + columname + ": " + str(missing))
    return check
# function to delete all data from graph database
def deleteGraphDbContent():
    gds = GraphDataScience("bolt://localhost:7687", auth=None)
    res = gds.run_cypher(
    """
    MATCH (n) DETACH DELETE n
    """
    )
    return res

In [ ]:
DATABASE_HOST="172.20.0.2"
#  vm.niif.cloud.bme.hu:18971 
# DATABASE_HOST="vm.niif.cloud.bme.hu"
# DATABASE_PORT=":18971/"
DATABASE_PORT=":5432/"
DATABASE_DATABASE="fabricexplorer"
DATABASE_USERNAME="hppoc"
DATABASE_PASSWORD="password"

# pio.renderers.default='notebook'

# list all user defined tables and schemas
# "SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'information_schema' AND schemaname != 'pg_catalog';"

#initializing resources
engine = create_engine('postgresql://'+DATABASE_USERNAME+
                       ':'+DATABASE_PASSWORD+'@'
                       +DATABASE_HOST
                       +DATABASE_PORT
                       +DATABASE_DATABASE)

# get all transactions from database
# txQuery = "SELECT * FROM transactions"
txQuery = "SELECT * FROM transactions ORDER BY id DESC limit 4000"
txDf = pd.read_sql(txQuery,con=engine)
# check if all data was recorded by explorer
continuityCheck(txDf,"blockid")
continuityCheck(txDf,"id")
blockQuery = "SELECT * FROM blocks"
blockDf = pd.read_sql(blockQuery,con=engine)

In [ ]:
# start tidying data, drop the transactions without readsets
txDf=txDf[txDf['read_set'].isna()==False]
# drop the ones committed by lifecycle chaincode
txDf=txDf[txDf['chaincodename'].str.contains('_lifecycle')==False]
# drop the config transactions
txDf=txDf[txDf['type'].str.contains('CONFIG')==False]
# convert validation code to string type
txDf['validation_code']=txDf['validation_code'].astype('string')
# drop a bunch of unneeded coloumns
timeDf=txDf.drop(columns=["txhash","chaincode_id","endorser_msp_id","type","channel_genesis_hash","envelope_signature","creator_id_bytes", "creator_nonce", "payload_extension","tx_response","payload_proposal_hash","endorser_id_bytes","endorser_signature", "network_name","chaincode_proposal_input","read_set","write_set"])
transactions = txDf.drop(columns=["txhash","chaincode_id","endorser_msp_id","type","channel_genesis_hash","envelope_signature","creator_id_bytes", "creator_nonce", "payload_extension","tx_response","payload_proposal_hash","endorser_id_bytes","endorser_signature", "network_name","chaincode_proposal_input",])
txDf=txDf.drop(columns=["txhash","chaincode_id","endorser_msp_id","type","createdt","channel_genesis_hash","envelope_signature","creator_id_bytes", "creator_nonce", "payload_extension","tx_response","payload_proposal_hash","endorser_id_bytes","endorser_signature", "network_name"])
# add default tx level in blocks to all transactions 
txDf['tx_level']=0
prev_id = 0
prev_lvl = 0
# sort transactions by id ascending
txDf = txDf.sort_values(by=['id'],ascending=True)
# shift the blockId up by one as previous blockid
txDf['prev_blockid']=txDf['blockid'].shift(1).fillna(0).astype(int)
# shift the tx level up by one as previous level
txDf['prev_lvl'] = txDf['tx_level'].shift(1).fillna(0).astype(int)
# determine tx level in blocks
txDf['tx_level'] = txDf.apply(blockLevelTxNum,axis=1)
# drop helper columns
txDf = txDf.drop(columns=['prev_lvl','prev_blockid'])
# drop lifecycle chaincodes
txDf = txDf[~((txDf['chaincodename'].str.contains('_lifecycle'))|(txDf['chaincodename'].str.contains('lscc')))]
# get all distinct chaincodes
distinct_cc = txDf['chaincodename'].unique()
continuityCheck(txDf,"blockid")

In [ ]:
# create rows from each read set
txDf = txDf.explode('read_set')
# create rows from each write set
txDf = txDf.explode('write_set')
# drop transactions with readsets and writest created by lifecycle chaincodes
txDf = txDf[~((txDf['read_set'].astype('string').str.contains("lscc"))|(txDf['read_set'].astype('string').str.contains("_lifecycle")))]
txDf = txDf[~((txDf['write_set'].astype('string').str.contains("lscc"))|(txDf['write_set'].astype('string').str.contains("_lifecycle")))]
# create separate dataframes for read and write keyaccesses
txDf_reads = txDf.drop(columns=['write_set'])
txDf_writes = txDf.drop(columns=['read_set'])
# add category information as access type
txDf_reads['access_type'] = 'READ'
txDf_writes['access_type'] = 'WRITE'
# find all written keys in write sets with regex
txDf_writes['keys']=txDf_writes['write_set'].astype('string').apply(lambda x: findKeys(x))
# write_set no longer needed, dropped
txDf_writes = txDf_writes.drop(columns=['write_set'])
# create rows from all keys in the list extracted from the write sets and rename column
txDf_writes = txDf_writes.explode('keys').rename(columns={'keys':'key'})
# drop null values (none should occur)
txDf_writes = txDf_writes[~txDf_writes['key'].isna()]
# extract keys and versions from the readsets' jsons
txDf_reads['read_set'] =txDf_reads['read_set'].apply(lambda x: x['set'])
txDf_reads = txDf_reads.explode('read_set')
txDf_reads = txDf_reads.dropna(subset=['read_set'])
txDf_reads['key'] =txDf_reads['read_set'].apply(lambda x: findKeys(str(x))[0])
# drop empty readsets
txDf_reads = txDf_reads.loc[(txDf_reads['read_set'].astype('string').str.contains('version', case=False, regex=False, na=False))]
txDf_reads['version_block'] =txDf_reads['read_set'].apply(lambda x:x['version']['block_num']['low'])
txDf_reads['version_tx'] =txDf_reads['read_set'].apply(lambda x: x['version']['tx_num']['low'])
# drop read_set, no longer needed
txDf_reads = txDf_reads.drop(columns=['read_set'])
# append writes to reads, creating dataframe with all keyaccesses
txDf = txDf_reads.append(txDf_writes).reset_index(drop=True).rename(columns={'id':'txid'})
# replace delimiting null character in composite keys
txDf['key'] = txDf['key'].str.replace("\\x00","||",regex=False)
# add block version information to write keyaccessess, based on the version they update the key to
txDf['version_block'] = txDf.apply(lambda x: x['version_block']if x['access_type']=='READ'else x['blockid'],axis=1).astype(int)
txDf['version_tx'] = txDf.apply(lambda x: x['version_tx']if x['access_type']=='READ'else x['tx_level'],axis=1).astype(int)
continuityCheck(txDf,'blockid')
txDf

## Keys and the number of reads on each of them
Seeing a lot of reads on a key might not be concerning, because if it is not written often, it likely won't cause a lot of MVCCs.

In [ ]:
# df containing reads only
txDf_reads=txDf[txDf['access_type']=='READ']
labels={"x":"Read keys","y":"Number of reads"}
# group by keys and count the occurrences
reads= txDf_reads.groupby(['key']).size()
reads = pd.DataFrame(reads)
reads.columns = [str(column) for column in reads.columns]
reads = reads.reset_index()
# sort keys by read amounts descending
reads = reads.sort_values(by=['0'], ascending=[False])
# plot the first 10 keys
fig = px.bar(reads.head(10), x='key', y='0', labels={'0':'reads'},title="Number of reads on each key")
#comment out this line to see the graph in log scale
# fig.update_yaxes(type='log')
fig

## Keys and the number of writes on each of them
Seeing a lot of writes on a key might not be concerning, because if it is not read often, it likely won't cause  a lot of MVCCs.

In [ ]:
# df containing writes only
txD_reads=txDf[txDf['access_type']=='WRITE']
labels={"x":"Written keys","y":"Number of writes"}
# group by keys and count the occurrences
writes = txD_reads.groupby(['key']).size()
writes = pd.DataFrame(writes)
writes.columns = [str(column) for column in writes.columns]
writes = writes.reset_index()
# sort keys by write amounts descending
writes = writes.sort_values(by=['0'], ascending=[False])
# plot the first 10 keys
fig = px.bar(writes.head(10), x='key', y='0', labels={'0':'writes'},title="Number of writes on each key")
#comment out this line to see the graph in log scale
# fig.update_yaxes(type='log')
fig

## Cumulative sum of valid txs - mvcc txs
If all goes well, this graph only shows a monotonically increasing line. However, it is great to visualize waves of mvcc conflicts, which will be visible as the slope of the graph will decrease, or even turn negative.

In [ ]:
# df containing transactions sorted by tx ids
transactions = transactions.sort_values(by=['id'], ascending=[True])
# add new column to df and assign values to transactions based on validation code
transactions['delta']= transactions['validation_code'].apply(lambda x: gradeValidation(x))
# extract keys from rwsets' jsons
transactions['read_keys'] = transactions['read_set'].apply(lambda x: findKeys(str(x)))
transactions['written_keys'] = transactions['write_set'].apply(lambda x: findKeys(str(x)))
# calculate the cumulative sum of the values assigned based on validation codes
transactions['delta_cumsum'] = transactions['delta'].cumsum()
# plot the data
fig = px.line(transactions.sort_values(by=['id'], ascending=[True]),
              x='id', y='delta_cumsum', hover_data=['read_keys', 'written_keys'],
              title="Cumulative sum of the number of Valid - MVCC transactions")
fig.update_xaxes(title_text='transaction id')
fig.update_yaxes(title_text='cumsum(valid-mvcc)')
#comment out this line to see the graph in log scale
# fig.update_yaxes(type='log')
fig

## Transactions grouped by validation codes
This graph just shows the number of transactions in each of the validation code groups.

In [ ]:
labels={"x":"Validation code","y":"Qty"}
# group transactions by validation codes, and calculate the size of each group
codes=transactions.groupby(['validation_code']).size().reset_index().rename(columns={0:'size'})
# sort rows by group size
codes = codes.sort_values(by=['size'], ascending=[False])
# plot the data
fig = px.bar(x=codes['validation_code'], y=codes['size'],labels=labels,title="Transactions by validation code")
#comment out this line to see the graph in linear scale
fig.update_yaxes(type='log')
fig

If your dataset contains large amounts of mvcc conflicts, please uncomment the cell below, and process the exported csv in the mvcc-finder notebook, that runs the kotlin kernel.

In [ ]:
# add called function info to df in a way that kotlin recoginizes them as strings
txDf['called_function']="'"+txDf['chaincode_proposal_input'].str.split(",", n = 1, expand = True)[0]+"'"
# sort by txid and access type ascending
df = txDf.sort_values(by=['txid', 'access_type'], ascending=[True, True])
# separate the called function from the proposal input
txDf['called_function']=txDf['chaincode_proposal_input'].str.split(",", n = 1, expand = True)[0]
# add new columns for counting mvccs caused by each keyaccess
df['mvcc_cause']=0
df['mvcc_caused_at']=""
df['mvcc_cause_found_for_tx']=False
df['mvcc_cause_for_tx']=0
df['mvcc_caused_at_for_tx']=""
# write df to disk as csv for processing with kotlin notebook
df.to_csv('data/txsmvccs_pre.csv')

**If you processed the dataframe with the kotlin notebook please dont run the next cell.**

In [ ]:
# sort by txid and access type ascending, drop old index
# df = pd.read_csv('data/txsmvccs_pre.csv')
# function to find the mvcc causing keyaccess
def findCause(df,row):
    # extract info about current row
    txId = row['txid']
    blockId = row['blockid']
    key = row['key']
    version_block = row['version_block']
    version_tx = row['version_tx']
    access = row['access_type']
    validation = row['validation_code']
    # if the access type is read and validation code is mvcc
    if validation=='MVCC_READ_CONFLICT' and access=='READ':
        # find all valid writes for the given key, in the interval between the keyaccess
        # and the block specified in the keyaccess' version
        subset = df[(
            (
                (df['blockid']>version_block) |
                ((df['blockid']==version_block) & (version_tx<df['version_tx']))
            ) &
            (df['key']==key)&
            (df['access_type']=='WRITE')&
            (df['validation_code']=='VALID')&
            (df['txid']<txId)
            )]
        # if the subset isn't empty
        if(len(subset)>0):
            # found the last valid write, +1 mvccs caused by it
            df['mvcc_cause'].iloc[[subset.tail(1).index[0]]]=df['mvcc_cause'].iloc[[subset.tail(1).index[0]]]+1
            df['mvcc_caused_at'].iloc[[subset.tail(1).index[0]]]=df['mvcc_caused_at'].iloc[[subset.tail(1).index[0]]] + str(txId) + ","
            # find out if any other key has been marked as mvcc causing in the transaction
            tx = df[(df['txid']==txId)&(df['mvcc_cause_found_for_tx']==True)]
            # if not mark this row as mvcc causing
            if(len(tx)==0):
                df['mvcc_cause_for_tx'].iloc[[subset.tail(1).index[0]]]=df['mvcc_cause_for_tx'].iloc[[subset.tail(1).index[0]]]+1
                df['mvcc_caused_at_for_tx'].iloc[[subset.tail(1).index[0]]]=df['mvcc_caused_at_for_tx'].iloc[[subset.tail(1).index[0]]] + str(txId) + ","
                df['mvcc_cause_found_for_tx'].loc[df['txid']==txId]=True
                
df = df.reset_index()
df.swifter.apply(lambda x: findCause(df,x),axis=1)
# write to disk as csv
df.to_csv('data/txsmvccs_post_no_gb.csv')
# group by keys and calculate the sum of mvccs they caused
df = df.groupby(['key']).agg(mvccs_caused=('mvcc_cause', 'sum')).reset_index()
# Keep rows where mvccs_caused > 0
df = df.loc[df['mvccs_caused'] > 0]
# sort rows by mvccs caused descending
df = df.sort_values(by=['mvccs_caused'], ascending=[False])
# os.system("rm data/txsmvccs_post.csv")
df.to_csv('data/txsmvccs_post.csv')
print('mvcc causes found total: ' + str(df['mvccs_caused'].sum()))

## Keys and the amount of MVCCs they caused
A key causes an mvcc conflict if it is read with a version number, that is different than the one in the current state of the blockchain.

In [ ]:
# read the csv containing the processed data
post_process = pd.read_csv("data/txsmvccs_post.csv")
# plot the first 10 keys
fig = px.bar(post_process.head(10), x='key', y='mvccs_caused',title="The number of MVCC conflicts caused by each key")
fig

In [ ]:
df = pd.read_csv('data/txsmvccs_post_no_gb.csv')
# group by keys and calculate the number of mvcc caused by each
df = df.groupby(['key']).agg(mvcc_cause_for_tx_sum=('mvcc_cause_for_tx', 'sum')).reset_index()
# sort rows by amount of mvcc caused descending (Z-A)
post_process = df.sort_values(by=['mvcc_cause_for_tx_sum'], ascending=[False])
print("Total number of mvccs found when limiting cause to 1/mvcc transaction: " + str(post_process['mvcc_cause_for_tx_sum'].sum()))

In [ ]:
fig = px.bar(post_process.head(10), x='key', y='mvcc_cause_for_tx_sum',title="The number of MVCC conflicts caused by each key, 1 cause/tx limit")
fig.update_yaxes(title="Quantity")
fig

In [ ]:
# read the processed dataframe containing the information about mvcc causing keys
post_process = pd.read_csv("data/txsmvccs_post_no_gb.csv")
# select mvcc causing keys
post_process = post_process[post_process["mvcc_cause"]>0]
# group data by the called function and calculate the sum mvccs caused by written keys in the function
post_process = post_process.groupby(['called_function']).agg(mvccs_caused=('mvcc_cause_for_tx', 'sum')).reset_index()
post_process

## Amount of MVCCS each transaction caused
With the help of the previous metric we can calculate the number of mvccs each transaction caused, by summing the previous number for all keys in a transaction's writeset.

In [ ]:
def to_1D(series):
    return pd.Series([x for _list in series for x in _list])
def getCausedAmount(keylist):
    amount = 0
    if not str(keylist)=='nan':
        for key in keylist:
            if(True):
                try:
                    amount += int(str(key).split('->')[1])
                except IndexError:
                    print(key)
    return amount
def getKeys(keylist):
    keys=[]
    if not str(keylist)=='nan':
        for key in keylist:
            if(len(key)>0):
                keys.append(key.split('->')[0])
    return keys
# Step: Select columns
transactions_tmp = transactions[['id', 'delta']]
# Step: Rename column
transactions_tmp = transactions_tmp.rename(columns={'id': 'txid'})
post_process = pd.read_csv("data/txsmvccs_post_no_gb.csv")
post_process['key_to_num_mvcc'] = post_process['key'].astype('string') + '->' + post_process['mvcc_cause'].astype('string')
# post_process['grade'] = post_process['validation_code'].apply(lambda x: gradeValidation(x))
post_process = post_process.groupby(['txid','access_type'])['key_to_num_mvcc'].apply(list)
post_process = post_process.reset_index(name ='keys_to_num_mvcc')
# Pivot dataframe from long to wide format using the variable column 'access_type' and the value column 'keys'
post_process = post_process.pivot(index='txid', columns='access_type', values='keys_to_num_mvcc').reset_index()
post_process.columns.name = ''
# Merge the dataframe with the dataframe containing all transactions and their grades on the txid 
post_process = pd.merge(post_process, transactions_tmp, on='txid', how='outer')
# Cumulative sum of the grades
post_process['delta_cumsum'] = post_process['delta'].cumsum()
post_process = post_process[(post_process['READ'].isna()==False)|(post_process['WRITE'].isna()==False)]
# Rename the columns
post_process = post_process.rename(columns={'READ': 'read_keys','WRITE': 'written_keys'})
post_process
# Extract the amount of mvccs each written key caused and calculate the sum of them to get the number of mvccs each transaction caused
post_process['mvccs_caused']=post_process['written_keys'].apply(lambda x: getCausedAmount(x))
# Extract the written keys from the written_key->number_of_mvccs_caused "datastructure"
post_process['written_keys']=post_process['written_keys'].apply(lambda x: getKeys(x))

In [ ]:
fig = px.line(post_process.sort_values(by=['txid'], ascending=[True]), x='txid', y='mvccs_caused', title='Amount of MVCCs each transaction caused')
fig

Reusing the previous calculation, and the one where each transaction was graded (-1 if it is mvcc or +1 if it is valid), we can better visualize where each mvcc causing write is in relation to the transactions they caused the confilct in.  

In [ ]:
fig = px.scatter(post_process, x='txid', y='delta_cumsum', size='mvccs_caused', title='Cumsum( #valid-#mvcc ) transactions, mark sized by mvccs caused', hover_data=['written_keys'])
fig.update_yaxes(title_text='cumsum( valid-mvcc )')
fig

In [ ]:
# copy the df with reads only
distDf = txDf_reads.copy()
# add a write distance column based on key version(block only) and current blockid
distDf['last_write_dist']=distDf['blockid']-distDf['version_block']
distDf

## Mean distance to last write of a key
If the block version number of the read is subtracted from the blockid of the transaction, we get the distance to the last write performed on the key. If this is averaged over all reads of the key, we get a kind of "hotness" indicator. The less "hot" the key is, the less likely it is to cause an MVCC.

In [ ]:
# group by keys and calculate the average distance to last write
dist = distDf.groupby(['key']).agg(last_write_dist_mean=('last_write_dist', 'mean')).reset_index()
# sort rows by average distance to last write ascending
dist = dist.sort_values(by=['last_write_dist_mean'], ascending=[True])
# plot the first 10 keys
fig = px.bar(dist.head(10), x='key', y='last_write_dist_mean',title="Mean distance to last write")
#comment out this line to see the graph in log scale
# fig.update_yaxes(type='log')
fig

## Transactions/second

In [ ]:
# df with transactions and their creation dates
tps = timeDf.drop(columns=["chaincodename","status","creator_msp_id",])
# sort rows by creation date ascending
tps = tps.sort_values(by=['createdt'], ascending=[True])
# reduce creation time resolution to seconds
tps['createdt'] = pd.to_datetime(tps['createdt'].dt.strftime('%Y-%m-%d %H:%M:%S'),format='%Y-%m-%d %H:%M:%S')
# calculate transactions created/second
tps = tps.groupby(['createdt']).size()
# plot the data
fig = px.line(x=tps.index,y=tps.values,labels={'x':'Time','y':'Tps'})
#comment out this line to see the graph in log scale
# fig.update_yaxes(type='log')
fig

General data about the transactions per second. "Count" here means the duration of the test, in seconds. 

In [ ]:
tps.describe()

# Transactions/second by validation code
The same data, but transaction count/second is colored by validation code. It helps visualize the amounts relative to eachother.

In [ ]:
# df with transactions and their creation dates
tps = timeDf.drop(columns=["chaincodename","status","creator_msp_id",])
# reduce creation time resolution to seconds
tps['createdt'] = pd.to_datetime(tps['createdt'].dt.strftime('%Y-%m-%d %H:%M:%S'),format='%Y-%m-%d %H:%M:%S')
# group by createdt and validation code
tps = tps.groupby(by=['createdt','validation_code'],as_index=False).size()
# Pivot df from long to wide format using the variable column 'validation_code' and the value column 'size'
tps = tps.pivot(index='createdt', columns='validation_code', values='size').reset_index()
# fill nan values with zeros
tps = tps.fillna(0)
# melt back to original shape
tps = tps.melt(id_vars=['createdt'],var_name='validation_code')
# sort rows by createdt ascending (A-Z)
tps = tps.sort_values(by=['createdt'], ascending=[True])
# plot the data
fig = px.area(tps, x='createdt', y='value', color='validation_code', color_discrete_sequence=px.colors.qualitative.G10[::-1])
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Tx/sec')
#comment out this line to see the graph in log scale
# fig.update_yaxes(type='log')
fig

## Number MVCCs/second
This graph shows how many transactions are validated with the code MVCC_READ_CONFLICT each second. 

In [ ]:
# df with transactions and their creation dates
mvccps = timeDf.drop(columns=["chaincodename","status","creator_msp_id",])
# sort rows by creation date ascending
mvccps = mvccps.sort_values(by=['createdt'], ascending=[True])
# reduce creation time resolution to seconds
mvccps['createdt'] = pd.to_datetime(mvccps['createdt'].dt.strftime('%Y-%m-%d %H:%M:%S'),format='%Y-%m-%d %H:%M:%S')
# keep transactions with validation code MVCC_READ_CONFLICT
mvccps = mvccps.loc[mvccps['validation_code'].isin(['MVCC_READ_CONFLICT'])]
# calculate number of mvcc transactions created/second
mvccps = mvccps.groupby(['createdt']).size()
# plot the data
fig = px.line(x=mvccps.index,y=mvccps.values,labels={'x':'Time','y':'MVCCS/sec'})
#comment out this line to see the graph in log scale
# fig.update_yaxes(type='log')
fig

## Read keys and the number of valid and mvcc transactions they occurred in
A key that was read as part of a transaction marked with mvcc might not be the cause of the mvcc, but can reveal patterns in the data.

In [ ]:
# group reads by keys and validation code, and calculate the size of each group
keycodedf = txDf_reads.groupby(by=['key','validation_code'],as_index=False).size()
# Pivot dataframe from long to wide format using the variable column 'validation_code' and the value column 'size'
# this is done so the data can be sorted by mvccs descending
keycodedf = keycodedf.pivot(index='key', columns='validation_code', values='size').reset_index()
# drop endorsement policy failure column
if 'ENDORSEMENT_POLICY_FAILURE' in keycodedf.columns:
    keycodedf = keycodedf.drop(columns=['ENDORSEMENT_POLICY_FAILURE'])
# sort by MVCC_READ_CONFLICT descending, keep the first 10
keycodedf = keycodedf.sort_values(by=['MVCC_READ_CONFLICT'], ascending=[False]).head(10)
# return to original shape so it can be plotted
keycodedf = keycodedf.melt(id_vars=['key'],var_name='validation_code')
# fill nans with 0, as they were before transformations
keycodedf = keycodedf.fillna(0)
# plot the data
labels={"key":"Key","value":"Occurrence in transactions"}
fig = px.bar(keycodedf, x='key', y='value', color='validation_code', barmode='group',labels=labels)
#comment out this line to see the graph in log scale
# fig.update_yaxes(type='log')
fig

## Keys and the ratio of valid/mvcc reads done on them.
If a key has a high ratio, it might be troublesome. It might not have caused the mvcc, but it might reveal patterns in the data.

In [ ]:
# previously created df with keys and their number of occurrences in each validation code group
# drop the endorsement policy failure group
ratio = txDf_reads.groupby(['key','validation_code']).size().reset_index().rename(columns={0:'size'})
ratio = ratio.loc[~(ratio['validation_code'].isin(['ENDORSEMENT_POLICY_FAILURE']))]
# pivot the table on validation codes
ratio = ratio.pivot_table(index=['key'], 
            columns=['validation_code'], values='size').fillna(0)
# calculate the ratio of the number of occurrences of each key in mvcc transactions/valid transactions
ratio['ratio']=ratio['MVCC_READ_CONFLICT']/ratio['VALID']
# drop rows where ratio is >= infinity  or <= 0
ratio = ratio.loc[~((ratio['ratio'] >= np.inf) | (ratio['ratio'] <= 0))]
# sort by the ratio descending
ratio = ratio.reset_index()
ratio = ratio.sort_values(by=['ratio'], ascending=[False])
# plot the first 10
fig = px.bar(ratio.head(10), x='key', y='ratio',title='Ratio of MVCC/Valid')
#comment out this line to see the graph in log scale
# fig.update_yaxes(type='log')
fig

# MVCC analysis for each asset type
Composite keys can be used in HLF to represent more complex data strutures. Below keys are grouped by asset types (the first part of the composite key by convention), and MVCC statistics are calculated for each type. This can show if a particular datatype is prone to be part of transactions with MVCC conflicts.

In [ ]:
ctxDf_reads=txDf_reads.copy()
ctxDf_reads["asset_type"]=ctxDf_reads["key"].apply(lambda x: getAssetType(x))
# Step: Group by asset_type, validation_code and calculate new column(s)
valcodesperasset = ctxDf_reads.groupby(['asset_type', 'validation_code']).agg(size=('validation_code', 'size')).reset_index()

## Validation codes by asset type in read and write sets
Not sure how useful this is...

In [ ]:
fig = px.bar(valcodesperasset, x='asset_type', y='size', facet_row='validation_code', facet_row_spacing=0.35,title="Validation code by asset type in read sets")
#comment out this line to see the graph in linear scale
fig.update_yaxes(type='log')
fig

In [ ]:
ctxD_reads=txD_reads.copy()
ctxD_reads["asset_type"]=ctxD_reads["key"].apply(lambda x: getAssetType(x))
# Step: Group by asset_type, validation_code and calculate new column(s)
valcodesperasset = ctxD_reads.groupby(['asset_type', 'validation_code']).agg(amount=('validation_code', 'size')).reset_index()

In [ ]:
fig = px.bar(valcodesperasset, x='asset_type', y='amount', facet_row='validation_code', facet_row_spacing=0.35,title="Validation code by asset type in write sets")
#comment out this line to see the graph in linear scale
fig.update_yaxes(type='log')
fig

In [ ]:
df = ctxDf_reads.copy()
# group by asset type and validation code and calculate the size of each group
df = df.groupby(['asset_type', 'validation_code']).agg(validation_code_size=('validation_code', 'size')).reset_index()
ratio = df.loc[~(df['validation_code'].isin(['ENDORSEMENT_POLICY_FAILURE']))]
ratio = ratio.pivot_table(index=['asset_type'], 
            columns=['validation_code'], values='validation_code_size').fillna(0)
# calculate the ratio of the number of occurrences of each key in mvcc transactions/valid transactions
ratio['ratio']=ratio['MVCC_READ_CONFLICT']/ratio['VALID']
# drop rows where ratio is >= infinity  or <= 0
ratio = ratio.loc[(~(ratio['ratio'] >= np.inf) | (ratio['ratio'] <= 0))]
# sort by the ratio descending
ratio = ratio.reset_index()
ratio = ratio.sort_values(by=['ratio'], ascending=[False])
ratio
# plot the first 10
fig = px.bar(ratio.head(10), x='asset_type', y='ratio',title='Ratio of MVCC/Valid by asset types in reads')
#comment out this line to see the graph in log scale
# fig.update_yaxes(type='log')
fig

In [ ]:
df = ctxD_reads.copy()
# group by asset type and validation code and calculate the size of each group
df = df.groupby(['asset_type', 'validation_code']).agg(validation_code_size=('validation_code', 'size')).reset_index()
ratio = df.loc[~(df['validation_code'].isin(['ENDORSEMENT_POLICY_FAILURE']))]
ratio = ratio.pivot_table(index=['asset_type'], 
            columns=['validation_code'], values='validation_code_size').fillna(0)
# calculate the ratio of the number of occurrences of each key in mvcc transactions/valid transactions
ratio['ratio']=ratio['MVCC_READ_CONFLICT']/ratio['VALID']
# drop rows where ratio is >= infinity  or <= 0
ratio = ratio.loc[~((ratio['ratio'] >= np.inf) | (ratio['ratio'] <= 0))]
# sort by the ratio descending
ratio = ratio.reset_index()
ratio = ratio.sort_values(by=['ratio'], ascending=[False])
ratio
# plot the first 10
fig = px.bar(ratio.head(10), x='asset_type', y='ratio',title='Ratio of MVCC/Valid by asset types in writes')
# comment out this line to see the graph in log scale
# fig.update_yaxes(type='log')
fig

## Sum of MVCCs caused by each asset type
Because we previously calculated the amount of mvccs each write caused, now these keys can be grouped by asset types and the mvcc amount can be summed. This will show if a particular asset type is more problematic and might require chaincode changes to improve the goodput of the network. 

In [ ]:
mvccs_by_type = pd.read_csv("data/txsmvccs_post.csv")
# get the asset types from the keys
mvccs_by_type["asset_type"]=mvccs_by_type['key'].apply(lambda x: getAssetType(x))
# Change data type of asset_type to String/Text for groupby to work correctly
mvccs_by_type['asset_type'] = mvccs_by_type['asset_type'].astype('string')
# Group by asset types and calculate the sum of mvccs they each caused
mvccs_by_type = mvccs_by_type.groupby(['asset_type']).agg(mvccs_caused_sum=('mvccs_caused', 'sum')).reset_index()
# Sort rows by mvccs_caused_sum ascending (A-Z)
mvccs_by_type = mvccs_by_type.sort_values(by=['mvccs_caused_sum'], ascending=[False])
fig = px.pie(mvccs_by_type.head(10), names='asset_type', values='mvccs_caused_sum', title='Percentage of total MVCCs caused by asset type')
fig

In [ ]:
fig = px.bar(mvccs_by_type, x='asset_type', y='mvccs_caused_sum',title="Total number of MVCCs caused by asset type")
fig.update_xaxes(title_text='Asset type')
fig.update_yaxes(title_text='Total mvccs caused')
fig

In [ ]:
# all user defined tables in the explorer db
query = "SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'information_schema' AND schemaname != 'pg_catalog';"
idkdf = pd.read_sql(query,con=engine)
idkdf

In [ ]:
df = pd.read_csv('data/txsmvccs_post_no_gb.csv')
# Step: Keep rows where (validation_code is one of: MVCC_READ_CONFLICT) and (mvcc_cause_found_for_tx is False)
df = df.loc[(df['validation_code'].isin(['MVCC_READ_CONFLICT'])) & (df['mvcc_cause_found_for_tx'] == False) & (df['access_type']=='READ') & (df['version_block']>df['blockid'].min())]

# Step: Group by and aggregate
df = df.groupby(['txid']).agg(txid_size=('txid', 'size')).reset_index()

df

In [ ]:
df = pd.read_csv('data/txsmvccs_post_no_gb.csv')
# group by keys and calculate the number of mvcc caused by each
df = df.groupby(['key']).agg(mvcc_cause_for_tx_sum=('mvcc_cause_for_tx', 'sum')).reset_index()
# sort rows by amount of mvcc caused descending (Z-A)
types = df.sort_values(by=['mvcc_cause_for_tx_sum'], ascending=[False])
types['asset_type']=types['key'].apply(lambda x:getAssetType(x))
types = types.groupby(['asset_type']).agg(mvcc_cause_for_tx_sum=('mvcc_cause_for_tx_sum', 'sum')).reset_index()
types = types.loc[types['mvcc_cause_for_tx_sum']>0].sort_values(by='mvcc_cause_for_tx_sum',ascending=False)
fig = px.pie(types.head(10), values='mvcc_cause_for_tx_sum', names='asset_type',title='Percentage of total MVCCs caused by asset type, 1 cause/tx limit')
fig

In [ ]:
fig = px.bar(types, x='asset_type', y='mvcc_cause_for_tx_sum', title='MVCC conflicts caused by each asset type, 1 cause/tx limit')
fig.update_yaxes(title_text='Quantity')
fig.update_xaxes(title_text='Asset type')
fig

In [ ]:
df = pd.read_csv('data/txsmvccs_post_no_gb.csv')
# Step: Keep rows where (validation_code is one of: MVCC_READ_CONFLICT) and (mvcc_cause_found_for_tx is False)
df = df.loc[(df['validation_code'].isin(['MVCC_READ_CONFLICT'])) & (df['mvcc_cause_found_for_tx'] == False)]
# Step: Group by and aggregate
df = df.groupby(['txid']).agg(key_size=('key', 'size')).reset_index().rename(columns={'key_size':'keyaccesses'})
print('MVCC transactions for which no cause is found: ' + str(len(df)))
df

In [ ]:
df = pd.read_csv('data/txsmvccs_post_no_gb.csv')
df['asset_type'] = df['key'].apply(lambda x: getAssetType(x))
df = df.groupby(['asset_type','mvcc_cause_for_tx']).size().reset_index().rename(columns={0:'size'})
df.set_index('asset_type',inplace=True)
causers = df['mvcc_cause_for_tx']*df['size']
causers = causers.groupby('asset_type').sum()
causers = causers.loc[causers > 0]
df = df.loc[df.index.isin(causers.index)]
df = df.loc[df.mvcc_cause_for_tx>0]
fig = px.histogram(df, x='mvcc_cause_for_tx', y='size', facet_row=df.index, nbins=df['mvcc_cause_for_tx'].max(), facet_row_spacing=0.08, 
                   height=len(causers)*250, title='Distribution of the number of MVCC conflicts caused by asset type per transaction')
fig.update_xaxes(title_text='Number of conflicts caused per transaction')
fig.update_yaxes(title_text='Quantity of txs')
fig

In [ ]:
df = pd.read_csv('data/txsmvccs_post_no_gb.csv')
df = df.loc[~df['mvcc_caused_at_for_tx'].isna()]
mvcc_txs = df['mvcc_caused_at_for_tx'].str.split(',').apply(lambda x: x[:-1])
mvcc_txs = np.concatenate(mvcc_txs.values).ravel().astype(int)
mvcc_txs = transactions.loc[transactions['id'].isin(mvcc_txs)]
mvcc_txs = mvcc_txs[['id','blockid','createdt', 'validation_code', 'read_keys', 'written_keys']]
mvcc_txs.to_csv('data/mvcc_txs.csv')
df = df[['txid','blockid','key','validation_code', 'version_block','version_tx','called_function','mvcc_cause_for_tx', 'mvcc_caused_at_for_tx']]
# df['mvcc_caused_at_for_tx'] = df['mvcc_caused_at_for_tx'].str.split(',').apply(lambda x: x[:-1])
df.to_csv('data/mvcc_keys.csv')
df

## Network analysis
Transactions or keys can be displayed as Nodes and different relationships between them can be represented with edges to gain more insight about the data access conflicts. 

### Database setup
The following cells initialize the neo4j database that was started with the script.

In [ ]:
deleteGraphDbContent()

## load mvcc keys to graph database
gds = GraphDataScience("bolt://localhost:7687", auth=None)
queryres = gds.run_cypher(
"""
LOAD CSV WITH HEADERS FROM 'file:///mvcc_keys.csv'
    AS row
    CREATE (k:Key{
        txid:row.txid,
        blockid:row.blockid,
        key:row.key,
        validation_code:row.validation_code,
        version_block:row.version_block,
        version_tx:row.version_tx,
        called_function:row.called_function,
        mvcc_cause:row.mvcc_cause_for_tx,
        mvcc_caused_at:row.mvcc_caused_at_for_tx
        })
"""
)
queryres

In [ ]:
## load transactions affected by mvcc conflicts into neo4j
gds = GraphDataScience("bolt://localhost:7687", auth=None)
queryres = gds.run_cypher(
"""
LOAD CSV WITH HEADERS FROM 'file:///mvcc_txs.csv'
    AS row
    CREATE (tx:Transaction{
        txid:row.id,
        blockid:row.blockid,
        createdt:row.createdt,
        read_keys:row.read_keys,
        written_keys:row.written_keys
        })
"""
)
queryres

In [ ]:
## transform array like list into array
gds = GraphDataScience("bolt://localhost:7687", auth=None)
queryres = gds.run_cypher(
"""
MATCH (k:Key) 
SET k.mvcc_caused_at = tail(reverse(split(k.mvcc_caused_at,','))) 
RETURN k
"""
)
queryres

In [ ]:
## create relationships between the mvcc causing keys and the transactions
gds = GraphDataScience("bolt://localhost:7687", auth=None)
queryres = gds.run_cypher(
"""
MATCH (k:Key)
UNWIND k.mvcc_caused_at AS txid 
MATCH (tx:Transaction) 
WHERE tx.txid = txid 
CREATE (k)-[r:CAUSES_MVCC]->(tx) 
RETURN type(r)
"""
)
queryres

### Post initialization
The database has been initialized if you have run the couple cells above this. To view the graph, open this [link](http://localhost:7474/browser/), select "No Authentication" and connect. After this you will be able to query the database with cypher, the query language of neo4j. To see the whole graph (max 300 nodes are displayed by default) run the following query:
```
MATCH (k:Key)-[r:CAUSES_MVCC]->(tx:Transaction) 
RETURN k, tx, COUNT(r) as n 
ORDER BY n DESC
```
This will display a graph with 2 types of nodes: key, transaction. The keys are written keys that caused mvcc conflicts, while the transactions are all invalidated ones with the code MVCC_READ_CONFLICT. The keys are connected by directed edges to the transactions that are invalidated because of them. If a node is clicked on, additional information can be seen on the right hand side panel. As mentioned above, the graph browser will only display 300 nodes by default. For this reason the results are ordered by the number of CAUSES_MVCC relationships in descending order.

In [ ]:
deleteGraphDbContent()
## load mvcc keys to graph database
gds = GraphDataScience("bolt://localhost:7687", auth=None)
df = pd.read_csv('data/txsmvccs_post_no_gb.csv')
# print(df.isnull().any())
df = df.fillna('-',axis=1)
# print(df.isnull().any())
df.to_csv('data/txsmvccs_post_no_gb_no_null.csv')
queryres = gds.run_cypher(
"""
LOAD CSV WITH HEADERS FROM 'file:///txsmvccs_post_no_gb_no_null.csv'
    AS row
    CREATE (k:Key{
        txid:row.txid,
        blockid:row.blockid,
        key:row.key,
        validation_code:row.validation_code,
        version_block:row.version_block,
        version_tx:row.version_tx,
        access_type:row.access_type,
        called_function:row.called_function,
        mvcc_cause:row.mvcc_cause_for_tx,
        mvcc_caused_at:row.mvcc_caused_at_for_tx
        })
"""
)
queryres


In [ ]:
queryres = gds.run_cypher(
"""
MATCH (k:Key) MERGE (f:Function{name:k.called_function}) RETURN f
"""
)
queryres

In [ ]:
queryres = gds.run_cypher(
"""
MATCH (f:Function), (k:Key)
WHERE f.name = k.called_function
AND k.access_type = "READ"
CREATE (f)-[r:READS{version_block:k.version_block,version_tx:k.version_tx}]->(k)
RETURN f,r,k
"""
)
queryres

In [ ]:
queryres = gds.run_cypher(
"""
MATCH (f:Function), (k:Key)
WHERE f.name = k.called_function
AND k.access_type = "WRITE"
CREATE (f)-[r:WRITES{version_block:k.version_block,version_tx:k.version_tx}]->(k)
RETURN f,r,k
"""
)
queryres

In [ ]:
queryres = gds.run_cypher(
"""
MATCH (k:Key)
MERGE (a:AssetType{type: split(k.key,"||")[1]})
RETURN a
"""
)
queryres

In [ ]:
queryres = gds.run_cypher(
"""
MATCH (k:Key),(a:AssetType)
WHERE split(k.key,"||")[1] = a.type
CREATE (k)-[r:IS_OF_TYPE]->(a)
RETURN type(r)
"""
)
queryres

In [ ]:
queryres = gds.run_cypher(
"""
MATCH (f:Function)-[r:READS]->(k:Key)-[IS_OF_TYPE]->(a:AssetType)
MERGE (f)-[rt:READS_TYPE]->(a)
RETURN type(rt), f.name,k.key,a.type
"""
)
queryres

In [ ]:
queryres = gds.run_cypher(
"""
MATCH (f:Function)-[r:WRITES]->(k:Key)-[IS_OF_TYPE]->(a:AssetType)
MERGE (f)-[wt:WRITES_TYPE]->(a)
RETURN type(wt), f.name,k.key,a.type
"""
)
queryres

In [ ]:
queryres = gds.run_cypher(
"""
MATCH (f:Function)-[r:READS_TYPE|WRITES_TYPE]->(a:AssetType)
RETURN f.name,type(r), a.type
"""
)
queryres

In [ ]:
df1 = pd.read_csv('data/txsmvccs_post_no_gb.csv')
df1 = df1.loc[df1['mvcc_cause_for_tx']>0]
df1 = df1.groupby(['called_function']).agg(mvccs_caused=('mvcc_cause_for_tx', 'sum')).reset_index()
df2 = pd.read_csv('data/txsmvccs_post_no_gb.csv')
df2 = df2.loc[df2['validation_code']=='MVCC_READ_CONFLICT']
df2 = df2.groupby(['txid','called_function']).size().groupby('called_function').size().reset_index().rename(columns={0:'invalidated_with_mvcc'})
df = pd.merge(df1,df2,on='called_function',how='outer').fillna(0,axis=1)
df

In [ ]:
# import os
# os.system("rm mvcc-eda.html")
# os.system("jupyter nbconvert mvcc-eda.ipynb --to html")